In [ ]:
import polars as pl
import os
import chromadb
from tqdm import tqdm
import time
import duckdb

# client = chromadb.Client()
# verses_collection = client.get_or_create_collection("kjv_verses")
# print(verses_collection.count())
persist_client = chromadb.PersistentClient(r"C:\repos\KJV_Search_Tools\.chroma")
persist_verses_collection = persist_client.get_or_create_collection("kjv_verses")
print(persist_verses_collection.count())
connection = duckdb.connect(r"C:\repos\KJV_Search_Tools\data\comparisons.duckdb")

In [ ]:
batch_size = 100
number_results = 40000
for i in tqdm(
    range(0, persist_verses_collection.count(), batch_size),
    total=persist_verses_collection.count() // batch_size,
):
    data = persist_verses_collection.get(
        include=["embeddings"], offset=i, limit=batch_size
    )
    results = persist_verses_collection.query(data["embeddings"], include=["distances"], n_results=number_results)
    batch_results = {}
    print(results)
    # for i, query in enumerate(data["ids"]):
    #     ids = results["ids"][i]
    #     distances = results["distances"][i]
    #     d = {}
    #     for j, id in enumerate(ids):
    #         distance = distances[j]
    #         d[id] = distance
    #     batch_results[data["ids"][i]] = d
    # # insert results into db
    # print(batch_results.keys())

In [ ]:
# kjv = pl.read_csv(r"C:\repos\KJV_Search_Tools\data\bible_data_set.csv")
# kjv = (
#     kjv
#     .with_columns([
#         pl.col("text")
#         .str.len_chars()
#         .alias("length_chars"),
#         pl.col("text").str.strip_chars()
#     ])
#     .rename(
#         {
#             "book": "book_name",
#             "chapter": "chapter_number",
#             "verse": "verse_number",
#         }
#     )
# )

# book_num_df = (
#     kjv.select(pl.col("book_name"))
#     .group_by("book_name", maintain_order=True)
#     .agg()
#     .with_row_index("book_number")
#     .with_columns(pl.col("book_number") + pl.lit(1))
# )

# kjv = (
#     kjv.join(book_num_df, pl.col("book_name"))
#     .with_columns(
#         [
#             (
#                 pl.col("book_number").cast(str)
#                 + pl.lit("_")
#                 + pl.col("chapter_number").cast(str)
#                 + pl.lit("_")
#                 + pl.col("verse_number").cast(str)
#             ).alias("id")
#         ]
#     )
#     .select(
#         [
#             pl.col("id"),
#             pl.col("citation"),
#             pl.col("book_name"),
#             pl.col("book_number"),
#             pl.col("chapter_number"),
#             pl.col("verse_number"),
#             pl.col("length_chars"),
#             pl.col("text"),
#         ]
#     )
# )

# print(kjv)
# kjv.write_csv(r"C:\repos\KJV_Search_Tools\data\kjv.csv")

# kjv = pl.read_csv(r"C:\repos\KJV_Search_Tools\data\kjv.csv")

# batch_size = 100
# for batch in (slices := tqdm(kjv.iter_slices(batch_size), total=len(kjv) / 100)):
#     slices.set_description(f"{batch.item(0,0)}")
#     ids = batch.select(pl.col("id")).to_series().to_list()
#     documents = batch.select(pl.col("text")).to_series().to_list()
#     metadatas = batch.select(
#         [
#             pl.col("citation"),
#             pl.col("book_name"),
#             pl.col("book_number"),
#             pl.col("chapter_number"),
#             pl.col("verse_number"),
#             pl.col("length_chars"),
#         ]
#     ).to_dicts()
#     persist_verses_collection.add(ids, None, metadatas, documents) # temporarily disabled in order to remove risk of overwriting or duplicates
# # 18m 16.5s for all 31102 verses in one go
# # 17m 35.2s for all verses in batches of 100
# print(persist_verses_collection.count())